# Read Messages texted
The Loop into person destination folder get your messages and concat in a CSV File

In [5]:

import os
import json
import csv

import  ftfy
from findJsonKeys import FindJsonKey

from populateFlatByJson import cleanText 

facebook_msg_folder=os.environ["FACE_DATA"]+"your_activity_across_facebook/messages/"

def transform(msg:dict,file:str)-> dict:
    msg["file"]=file
    if("content" in msg):
        msg["text"]=cleanText(msg["content"])
    if("sender_name" in msg):
        msg["sender_name"]=cleanText(msg["sender_name"])
        
    return msg

findKeys=FindJsonKey()

all_data=[]

for root,folders,files in os.walk(facebook_msg_folder):
    folder=root.replace(facebook_msg_folder,"")
    for file in [file for file in files if "message" in file]:
        with open(os.path.join(root,file)) as user_file:
            parsed_json = json.load(user_file)
            if("messages" not in parsed_json):
                continue
            findKeys.makeDecision(parsed_json)
            parsed_json = parsed_json["messages"]
            parsed_json = [transform(msg,folder) for msg in parsed_json]
            
            all_data += (parsed_json)

print(findKeys.keys())   

with open(os.path.join(os.path.dirname(facebook_msg_folder),"msg.json"), 'w', newline='', encoding='utf-8') as jsonFile:                
    
    parsed_json = [item for item in all_data]
    json.dump(parsed_json,jsonFile,indent=2)    

image"]
image"]["creation_timestamp"]
image"]["uri"]
is_pending"]
is_still_participant"]
joinable_mode"]
joinable_mode"]["link"]
joinable_mode"]["mode"]
magic_words"]
messages"]
messages"]["$L$"]["audio_files"]
messages"]["$L$"]["audio_files"]["$L$"]["creation_timestamp"]
messages"]["$L$"]["audio_files"]["$L$"]["uri"]
messages"]["$L$"]["call_duration"]
messages"]["$L$"]["content"]
messages"]["$L$"]["gifs"]
messages"]["$L$"]["gifs"]["$L$"]["uri"]
messages"]["$L$"]["ip"]
messages"]["$L$"]["is_geoblocked_for_viewer"]
messages"]["$L$"]["is_unsent"]
messages"]["$L$"]["missed"]
messages"]["$L$"]["photos"]
messages"]["$L$"]["photos"]["$L$"]["creation_timestamp"]
messages"]["$L$"]["photos"]["$L$"]["uri"]
messages"]["$L$"]["reactions"]
messages"]["$L$"]["reactions"]["$L$"]["actor"]
messages"]["$L$"]["reactions"]["$L$"]["reaction"]
messages"]["$L$"]["sender_name"]
messages"]["$L$"]["share"]
messages"]["$L$"]["share"]["link"]
messages"]["$L$"]["share"]["original_content_owner"]
messages"]["$L$"][